# LRP for clustering

In [1]:
%load_ext autoreload
%autoreload 2

In [155]:
import matplotlib.pyplot as plt
from sklearn import datasets
import mpl_toolkits.mplot3d 
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
import pandas as pd
from cxplain.neon import NeonKMeansExplainer

## Normal k-means

In [125]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
n_clusters = 3
num_features = 4

In [15]:
kmeans = KMeans(n_clusters=n_clusters, random_state=3).fit(X)
cluster_predictions = kmeans.predict(X)

In [144]:
cluster_centers = kmeans.cluster_centers_
cluster_centers.shape[0]

3

In [10]:
cluster_predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0])

In [ ]:
# Layer 0 150 x 4, Layer 2 150 x 3, Layer 3 150 x 1   
# General : Layer 0 num_obs x num_features, Layer 2 num_obs x num_clusters, Layer 3 num_obs x 1

In [90]:
index_actual = cluster_predictions[0]
index_0 = 0
index_observation = 0
center_actual = cluster_centers[index_actual]
center_0 = cluster_centers[index_0]
observation_i = X[index_observation]

In [91]:
# calculate weights vector for one cluster
weights_0 = 2 * (center_actual - center_0)
bias_0 =  np.linalg.norm(center_0, ord=2)**2 - np.linalg.norm(center_actual, ord=2)**2 
h_0 = weights_0.dot(observation_i) + bias_0

In [93]:
# calculate forward pass
centers_actual = np.vstack([center_actual] * n_clusters) # I need to subtract every cluster center from the actual one --> repeat n-Cluster times to do it all in one matrix subtraction 
weights = 2 * (centers_actual - cluster_centers)
bias = np.linalg.norm(cluster_centers, ord=2, axis=1)**2 - np.linalg.norm(centers_actual, ord=2, axis=1)**2
hidden_1 = weights.dot(observation_i) + bias
output = np.amin(np.delete(hidden_1, index_actual))

In [152]:
# calculate backward pass
# determine beta,
beta = 0.5
# relevance intermediate layer
relevance_intermediate = (np.exp(-beta * hidden_1) / (np.sum(np.exp(-beta * np.delete(hidden_1, index_actual))))) * output
# relevance input
centers_distance = weights / 4
centers_distance_wo_actual = np.delete(centers_distance, index_actual, axis=0)
hidden_wo_actual = np.delete(hidden_1, index_actual, axis=0)
contribution = np.multiply((np.vstack([observation_i] * (n_clusters - 1)) - centers_distance_wo_actual), np.vstack([hidden_wo_actual]* num_features).T)
sum_contribution = np.sum(contribution, axis=0)
relevance_intermediate_wo_actual = np.delete(relevance_intermediate, index_actual, axis=0)
cluster_contribution = np.multiply(np.vstack([relevance_intermediate_wo_actual] * num_features).T, (contribution / sum_contribution))
feature_relevances_2 = np.sum(cluster_contribution, axis=0)

In [135]:
rel = np.multiply((np.vstack([observation_i] * (n_clusters - 1)) - centers_distance_wo_actual), np.vstack([hidden_wo_actual] * num_features).T)
norm_rel = np.sum(rel, axis=0)
quotient = (rel / norm_rel) #* np.delete(relevance_intermediate, index_actual, axis=0)

In [129]:
relevances_wo = np.delete(relevance_intermediate, index_actual, axis=0)
summand = np.multiply(np.vstack([relevances_wo] * num_features).T, (rel / norm_rel))
np.sum(summand, axis=0)

array([3.61217025, 7.53545926, 4.16625167, 3.70579392])

In [228]:
neon = NeonKMeansExplainer(cluster_centers=cluster_centers, data=X, predictions=cluster_predictions)

In [229]:
neon.fit()

In [230]:
neon.is_fitted

True

In [231]:
neon_explanation = neon.explain()

In [232]:
neon_explanation.global_relevance

R1    1.795856
R2    1.787152
R3    1.770622
R4    1.729748
dtype: float64

In [233]:
network = neon._init_network(0)
beta = neon._get_beta()
beta

0.17456391396625548

In [234]:
network.forward(X[0])
network.output

11.671294713839742

In [235]:
network.backward(X[0], beta)

array([3.84030905, 3.90558126, 3.58400297, 3.34829464])

In [236]:
feature_relevances_2

array([3.45867224, 3.53642868, 3.15334397, 2.87255308])